<a href="https://colab.research.google.com/github/aekanun2020/2022-PSU-Data-Science-for-Tourism-Course1/blob/main/Quiz2-PySpark_DataScience_onColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
   .appName("Neural Network Model") \
   .config("spark.executor.memory", "3gb") \
   .getOrCreate()
   
sc = spark.sparkContext

In [ ]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [ ]:
! head -3 testfunnel.csv

source,isTrueDirect,sourceKeyword,medium,isVideoAd,fullVisitorId,visitId,date,newVisits,hitReferer,hitType,hitAction_type,hitNumber,hitHour,hitMin,timeMicroSec,v2ProductName,productListName,isClick,isImpression,sessionQualityDim,timeOnScreen,timeOnSite,totalTransactionRevenue
google,true,(not provided),organic,,4988612949713423910,1489617360,20170315,,,PAGE,0,28,15,46,608771,Google Accent Insulated Stainless Steel Bottle,Category,,true,,,1897,
google,true,(not provided),organic,,4988612949713423910,1489617360,20170315,,,PAGE,0,27,15,46,605064,Google Pocket Bluetooth Speaker,Category,,true,,,1897,


In [ ]:
! wget https://storage.googleapis.com/sharewebinar/testfunnel.csv

--2021-08-26 07:39:04--  https://storage.googleapis.com/sharewebinar/testfunnel.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.128, 142.251.2.128, 2607:f8b0:4023:c06::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1501012 (1.4M) [text/csv]
Saving to: ‘testfunnel.csv.6’

testfunnel.csv.6    100%[===================>]   1.43M  --.-KB/s    in 0.02s   

2021-08-26 07:39:04 (64.2 MB/s) - ‘testfunnel.csv.6’ saved [1501012/1501012]



In [ ]:
raw_df = spark.read.option('header','true')\
.option("inferSchema" , "true")\
.format('csv').load('./testfunnel.csv')

In [ ]:
raw_df.sample(0.001,seed=12).toPandas().transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12
source,(direct),(direct),google,google,google,google,(direct),(direct),(direct),google,(direct),google,google
isTrueDirect,None,None,None,None,None,None,True,True,True,None,None,True,None
sourceKeyword,None,None,(not provided),(not provided),(not provided),(not provided),None,None,None,(not provided),None,(not provided),(not provided)
medium,(none),(none),organic,organic,organic,organic,(none),(none),(none),organic,(none),organic,organic
isVideoAd,None,None,None,None,None,None,None,None,None,None,None,None,None
fullVisitorId,4990075466656363861,4990257633001131707,4990697502077570091,4991105026836415438,4991962803971844700,4992499749003799871,4992819126088253676,4993613447705442396,4993613447705442396,4994855995081050813,4995356988620593823,4995584940873803432,4995903322841177843
visitId,1480486355,1485540848,1494612328,1488300417,1475876346,1472235854,1476938325,1482297870,1483430590,1495711333,1481560042,1497463936,1493223582
date,20161129,20170127,20170512,20170228,20161007,20160826,20161019,20161220,20170103,20170525,20161212,20170614,20170426
newVisits,1,1,1,1,1,1,1,NaN,NaN,1,1,NaN,1
hitReferer,None,https://www.google.nl/,None,None,None,None,None,None,None,https://www.google.ie/,None,None,None


1. ธุรกิจต้องการ ML Model ไปใช้ในการทำนายว่า ลูกค้าที่กำลังเลือกดูสินค้าอยู่ จะซื้อหรือไม่ซื้อสินค้า (รวมถึงพฤติกรรมอื่นๆ ด้วย)

In [ ]:
raw_df.count()

9999

2. Data Understanding

In [ ]:
len(raw_df.columns)

24

In [ ]:
raw_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
source,9999,None,None,(direct),youtube.com
sourceKeyword,4042,None,None,(not provided),youtube merch
medium,9999,None,None,(none),referral
fullVisitorId,9999,4.5067648825526333E18,1.39995873491150899E18,49898505875812980,4995924842281653133
visitId,9999,1.4850613864552455E9,9020666.664979974,1470106850,1501649190
date,9999,2.016601000310031E7,4694.944823979139,20160801,20170801
newVisits,7269,1.0,0.0,1,1
hitReferer,1333,None,None,http://images.google.co.in/imgres?imgurl=https...,https://www.youtube.com/yt/about/press/
hitType,9999,None,None,EVENT,PAGE


In [ ]:
raw_df.printSchema()

root
 |-- source: string (nullable = true)
 |-- isTrueDirect: boolean (nullable = true)
 |-- sourceKeyword: string (nullable = true)
 |-- medium: string (nullable = true)
 |-- isVideoAd: boolean (nullable = true)
 |-- fullVisitorId: long (nullable = true)
 |-- visitId: integer (nullable = true)
 |-- date: integer (nullable = true)
 |-- newVisits: integer (nullable = true)
 |-- hitReferer: string (nullable = true)
 |-- hitType: string (nullable = true)
 |-- hitAction_type: integer (nullable = true)
 |-- hitNumber: integer (nullable = true)
 |-- hitHour: integer (nullable = true)
 |-- hitMin: integer (nullable = true)
 |-- timeMicroSec: integer (nullable = true)
 |-- v2ProductName: string (nullable = true)
 |-- productListName: string (nullable = true)
 |-- isClick: boolean (nullable = true)
 |-- isImpression: boolean (nullable = true)
 |-- sessionQualityDim: integer (nullable = true)
 |-- timeOnScreen: string (nullable = true)
 |-- timeOnSite: integer (nullable = true)
 |-- totalTransac

In [ ]:
raw_df.groupBy('isTrueDirect').count().show()

+------------+-----+
|isTrueDirect|count|
+------------+-----+
|        null| 6648|
|        true| 3351|
+------------+-----+



In [ ]:
from pyspark.sql import functions as sparkf

In [ ]:
selectedCol_df = raw_df\
.drop('isVideoAd').drop('isClick').drop('isImpression')\
.drop('sourceKeyword').drop('hitReferer').drop('timeOnScreen').drop('sessionQualityDim')

In [ ]:
selectedCol_df.count()

9999

In [ ]:
len(selectedCol_df.columns)

17

3. Data Preparation

In [ ]:
from pyspark.sql.types import *

In [ ]:
sparkf_NulltoFalse = sparkf.udf(lambda x: 'false' if x is None else x)

In [ ]:
selectedCol_df.withColumn('isTrueDirect',\
                          sparkf.col('isTrueDirect').cast(StringType()))\
.withColumn('isTrueDirect',\
           sparkf_NulltoFalse(sparkf.col('isTrueDirect')))\
.groupBy('isTrueDirect').count().show()

+------------+-----+
|isTrueDirect|count|
+------------+-----+
|       false| 6648|
|        true| 3351|
+------------+-----+



In [ ]:
sparkf_treatNewVisits = sparkf.udf(lambda x: 'true' if x == '1' else 'false')

In [ ]:
selectedCol_df.withColumn('newVisits',\
                          sparkf.col('newVisits').cast(StringType()))\
.withColumn('newVisits',sparkf_treatNewVisits(sparkf.col('newVisits')))\
.groupBy('newVisits').count().show()

+---------+-----+
|newVisits|count|
+---------+-----+
|    false| 2730|
|     true| 7269|
+---------+-----+



In [ ]:
sparkf_treatRevenue = sparkf.udf(lambda x: 0 if x == None else x)

In [ ]:
selectedCol_df.printSchema()

root
 |-- source: string (nullable = true)
 |-- isTrueDirect: boolean (nullable = true)
 |-- medium: string (nullable = true)
 |-- fullVisitorId: long (nullable = true)
 |-- visitId: integer (nullable = true)
 |-- date: integer (nullable = true)
 |-- newVisits: integer (nullable = true)
 |-- hitType: string (nullable = true)
 |-- hitAction_type: integer (nullable = true)
 |-- hitNumber: integer (nullable = true)
 |-- hitHour: integer (nullable = true)
 |-- hitMin: integer (nullable = true)
 |-- timeMicroSec: integer (nullable = true)
 |-- v2ProductName: string (nullable = true)
 |-- productListName: string (nullable = true)
 |-- timeOnSite: integer (nullable = true)
 |-- totalTransactionRevenue: integer (nullable = true)



In [ ]:
sparkf_createLabel = sparkf.udf(lambda x: 'buy' if x>0 else 'notBuy')

In [ ]:
crunched_df = selectedCol_df\
.withColumn('isTrueDirect',sparkf.col('isTrueDirect').cast(StringType()))\
.withColumn('isTrueDirect',sparkf_NulltoFalse(sparkf.col('isTrueDirect')))\
.withColumn('newVisits',sparkf.col('newVisits').cast(StringType()))\
.withColumn('newVisits',sparkf_treatNewVisits(sparkf.col('newVisits')))\
.withColumn('totalTransactionRevenue',sparkf_treatRevenue(sparkf.col('totalTransactionRevenue')))\
.withColumn('totalTransactionRevenue',sparkf.col('totalTransactionRevenue').cast(DoubleType()))\
.withColumn('label',sparkf_createLabel(sparkf.col('totalTransactionRevenue')))

In [ ]:
crunched_df.printSchema()

root
 |-- source: string (nullable = true)
 |-- isTrueDirect: string (nullable = true)
 |-- medium: string (nullable = true)
 |-- fullVisitorId: long (nullable = true)
 |-- visitId: integer (nullable = true)
 |-- date: integer (nullable = true)
 |-- newVisits: string (nullable = true)
 |-- hitType: string (nullable = true)
 |-- hitAction_type: integer (nullable = true)
 |-- hitNumber: integer (nullable = true)
 |-- hitHour: integer (nullable = true)
 |-- hitMin: integer (nullable = true)
 |-- timeMicroSec: integer (nullable = true)
 |-- v2ProductName: string (nullable = true)
 |-- productListName: string (nullable = true)
 |-- timeOnSite: integer (nullable = true)
 |-- totalTransactionRevenue: double (nullable = true)
 |-- label: string (nullable = true)



In [ ]:
crunched_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
source,9999,None,None,(direct),youtube.com
isTrueDirect,9999,None,None,false,true
medium,9999,None,None,(none),referral
fullVisitorId,9999,4.5067648825526333E18,1.39995873491150899E18,49898505875812980,4995924842281653133
visitId,9999,1.4850613864552455E9,9020666.664979974,1470106850,1501649190
date,9999,2.016601000310031E7,4694.944823979139,20160801,20170801
newVisits,9999,None,None,false,true
hitType,9999,None,None,EVENT,PAGE
hitAction_type,9999,0.13831383138313832,0.6864877640641197,0,6


In [ ]:
crunched_df.dropna().count()

8958

In [ ]:
final_df = crunched_df.dropna()

In [ ]:
final_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
source,8958,None,None,(direct),youtube.com
isTrueDirect,8958,None,None,false,true
medium,8958,None,None,(none),referral
fullVisitorId,8958,4.5213872161084938E18,1.37948897535059482E18,49898679051871719,4995924842281653133
visitId,8958,1.484833213452668E9,8954199.880686603,1470106850,1501649190
date,8958,2.01658726431123E7,4700.377098126274,20160801,20170801
newVisits,8958,None,None,false,true
hitType,8958,None,None,EVENT,PAGE
hitAction_type,8958,0.15438713998660417,0.7235710402472797,0,6


In [ ]:
final_df.groupBy('label').count().show()

+------+-----+
| label|count|
+------+-----+
|   buy|  707|
|notBuy| 8251|
+------+-----+



4. Modeling

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [ ]:
final_df.columns

['source',
 'isTrueDirect',
 'medium',
 'fullVisitorId',
 'visitId',
 'date',
 'newVisits',
 'hitType',
 'hitAction_type',
 'hitNumber',
 'hitHour',
 'hitMin',
 'timeMicroSec',
 'v2ProductName',
 'productListName',
 'timeOnSite',
 'totalTransactionRevenue',
 'label']

In [ ]:
final_df.filter(sparkf.col('visitId') == '1470106850')\
.orderBy('hitNumber')\
.select(['fullVisitorId','visitId','hitHour','hitMin','v2ProductName','hitAction_type','label'])\
.show(100)

+-------------------+----------+-------+------+--------------------+--------------+-----+
|      fullVisitorId|   visitId|hitHour|hitMin|       v2ProductName|hitAction_type|label|
+-------------------+----------+-------+------+--------------------+--------------+-----+
|4993485206334150199|1470106850|     20|     1|Women's YouTube S...|             0|  buy|
|4993485206334150199|1470106850|     20|     1| YouTube Trucker Hat|             0|  buy|
|4993485206334150199|1470106850|     20|     1|YouTube Women's R...|             0|  buy|
|4993485206334150199|1470106850|     20|     1|YouTube Women's S...|             0|  buy|
|4993485206334150199|1470106850|     20|     1|22 oz YouTube Bot...|             0|  buy|
|4993485206334150199|1470106850|     20|     1|YouTube Men's Sho...|             0|  buy|
|4993485206334150199|1470106850|     20|     1|YouTube Men's Lon...|             0|  buy|
|4993485206334150199|1470106850|     20|     1|YouTube Men's 3/4...|             0|  buy|
|499348520

In [ ]:
final_df.filter(sparkf.col('label') == 'buy').orderBy(['visitId','timeMicroSec']).limit(10).toPandas().transpose()

,0,1,2,3,4,5,6,7,8,9
source,youtube.com,youtube.com,youtube.com,youtube.com,youtube.com,youtube.com,youtube.com,youtube.com,youtube.com,youtube.com
isTrueDirect,false,false,false,false,false,false,false,false,false,false
medium,referral,referral,referral,referral,referral,referral,referral,referral,referral,referral
fullVisitorId,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199
visitId,1470106850,1470106850,1470106850,1470106850,1470106850,1470106850,1470106850,1470106850,1470106850,1470106850
date,20160801,20160801,20160801,20160801,20160801,20160801,20160801,20160801,20160801,20160801
newVisits,true,true,true,true,true,true,true,true,true,true
hitType,PAGE,PAGE,PAGE,PAGE,PAGE,PAGE,PAGE,PAGE,PAGE,PAGE
hitAction_type,0,0,0,0,0,0,0,0,0,0
hitNumber,2,2,2,2,2,2,2,2,2,2


In [ ]:
final_df.orderBy(['visitId','timeMicroSec']).limit(10).toPandas().transpose()

,0,1,2,3,4,5,6,7,8,9
source,youtube.com,youtube.com,youtube.com,youtube.com,youtube.com,youtube.com,youtube.com,youtube.com,youtube.com,youtube.com
isTrueDirect,false,false,false,false,false,false,false,false,false,false
medium,referral,referral,referral,referral,referral,referral,referral,referral,referral,referral
fullVisitorId,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199,4993485206334150199
visitId,1470106850,1470106850,1470106850,1470106850,1470106850,1470106850,1470106850,1470106850,1470106850,1470106850
date,20160801,20160801,20160801,20160801,20160801,20160801,20160801,20160801,20160801,20160801
newVisits,true,true,true,true,true,true,true,true,true,true
hitType,PAGE,PAGE,PAGE,PAGE,PAGE,PAGE,PAGE,PAGE,PAGE,PAGE
hitAction_type,0,0,0,0,0,0,0,0,0,0
hitNumber,2,2,2,2,2,2,2,2,2,2


In [ ]:
sourceIndexer = StringIndexer(inputCol='source', outputCol='sourceIndexed',
    handleInvalid='keep')
sourceOneHotEncoder = OneHotEncoder(inputCol='sourceIndexed', outputCol='sourceVec')

In [ ]:
isTrueDirectIndexer = StringIndexer(inputCol='isTrueDirect', outputCol='isTrueDirectIndexed',
    handleInvalid='keep')
isTrueDirectOneHotEncoder = OneHotEncoder(inputCol='isTrueDirectIndexed', outputCol='isTrueDirectVec')

In [ ]:
mediumIndexer = StringIndexer(inputCol='medium', outputCol='mediumIndexed',
    handleInvalid='keep')
mediumOneHotEncoder = OneHotEncoder(inputCol='mediumIndexed', outputCol='mediumVec')

In [ ]:
newVisitsIndexer = StringIndexer(inputCol='newVisits', outputCol='newVisitsIndexed',
    handleInvalid='keep')
newVisitsOneHotEncoder = OneHotEncoder(inputCol='newVisitsIndexed', outputCol='newVisitsVec')

In [ ]:
hitTypeIndexer = StringIndexer(inputCol='hitType', outputCol='hitTypeIndexed',
    handleInvalid='keep')
hitTypeOneHotEncoder = OneHotEncoder(inputCol='hitTypeIndexed', outputCol='hitTypeVec')

In [ ]:
v2ProductNameIndexer = StringIndexer(inputCol='v2ProductName', outputCol='v2ProductNameIndexed',
    handleInvalid='keep')
v2ProductNameOneHotEncoder = OneHotEncoder(inputCol='v2ProductNameIndexed', outputCol='v2ProductNameVec')

In [ ]:
productListNameIndexer = StringIndexer(inputCol='productListName', outputCol='productListNameIndexed',
    handleInvalid='keep')
productListNameOneHotEncoder = OneHotEncoder(inputCol='productListNameIndexed', outputCol='productListNameVec')

In [ ]:
labelIndexer = StringIndexer(inputCol='label', outputCol='labelIndexed',
    handleInvalid='keep')
#labelOneHotEncoder = OneHotEncoder(inputCol='labelIndexed', outputCol='labelVec')

In [ ]:
from pyspark.mllib.linalg import Vectors

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
featureAssembler = VectorAssembler(inputCols=['sourceVec'\
                                             ,'isTrueDirectVec'\
                                             ,'mediumVec'\
                                             ,'newVisitsVec'\
                                             ,'hitTypeVec'\
                                             ,'v2ProductNameVec'\
                                             ,'productListNameVec'\
                                             ,'hitNumber'\
                                             ,'timeMicroSec'], outputCol='features')

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

In [ ]:
algo = DecisionTreeClassifier(labelCol='labelIndexed',featuresCol='features')

In [ ]:
pipeline_train = Pipeline().setStages([sourceIndexer,sourceOneHotEncoder,isTrueDirectIndexer,isTrueDirectOneHotEncoder\
                     ,mediumIndexer,mediumOneHotEncoder,newVisitsIndexer,newVisitsOneHotEncoder\
                     ,hitTypeIndexer,hitTypeOneHotEncoder,v2ProductNameIndexer,v2ProductNameOneHotEncoder\
                     ,productListNameIndexer,productListNameOneHotEncoder\
                                       ,labelIndexer\
                                       ,featureAssembler,algo])

In [ ]:
training_df, testing_df = final_df.randomSplit(weights = [0.80, 0.20], seed = 13)

In [ ]:
model = pipeline_train.fit(training_df)

In [ ]:
model

PipelineModel_fce4c1b9cdbb

5. Model Evaluation

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [ ]:
result_df = model.transform(testing_df)
evaluator_RF = MulticlassClassificationEvaluator(predictionCol="prediction",\
                                              labelCol="labelIndexed", metricName='accuracy')
print('evaluation','====>',evaluator_RF.evaluate(result_df)*100)

evaluation ====> 93.78042184964845


In [ ]:
result_df

DataFrame[source: string, isTrueDirect: string, medium: string, fullVisitorId: bigint, visitId: int, date: int, newVisits: string, hitType: string, hitAction_type: int, hitNumber: int, hitHour: int, hitMin: int, timeMicroSec: int, v2ProductName: string, productListName: string, timeOnSite: int, totalTransactionRevenue: double, label: string, sourceIndexed: double, sourceVec: vector, isTrueDirectIndexed: double, isTrueDirectVec: vector, mediumIndexed: double, mediumVec: vector, newVisitsIndexed: double, newVisitsVec: vector, hitTypeIndexed: double, hitTypeVec: vector, v2ProductNameIndexed: double, v2ProductNameVec: vector, productListNameIndexed: double, productListNameVec: vector, labelIndexed: double, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [ ]:
result_df.groupBy('labelIndexed').count().show()

+------------+-----+
|labelIndexed|count|
+------------+-----+
|         0.0| 1701|
|         1.0|  148|
+------------+-----+



In [ ]:
result_df.filter(sparkf.col('labelIndexed') == sparkf.col('prediction'))\
.select(['labelIndexed','prediction']).groupBy('labelIndexed').count().show()

+------------+-----+
|labelIndexed|count|
+------------+-----+
|         0.0| 1693|
|         1.0|   41|
+------------+-----+



In [ ]:
result_df.filter(sparkf.col('labelIndexed') != sparkf.col('prediction'))\
.select(['labelIndexed','prediction']).groupBy('labelIndexed').count().show()

+------------+-----+
|labelIndexed|count|
+------------+-----+
|         0.0|    8|
|         1.0|  107|
+------------+-----+



In [ ]:
Recall = 2782/14015

In [ ]:
Recall

0.19850160542276132